# Experiment 3 – CNN


## Imports + Device

In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


CUDA available: True
GPU: Tesla T4


In [3]:
from google.colab import files
files.upload()


Saving cats_dogs.zip to cats_dogs.zip


In [1]:
!unzip cats_dogs.zip


Streaming output truncated to the last 5000 lines.
  inflating: cats_dogs/val/cats/10010.jpg  
  inflating: cats_dogs/val/cats/10013.jpg  
  inflating: cats_dogs/val/cats/10021.jpg  
  inflating: cats_dogs/val/cats/10026.jpg  
  inflating: cats_dogs/val/cats/10035.jpg  
  inflating: cats_dogs/val/cats/10040.jpg  
  inflating: cats_dogs/val/cats/1005.jpg  
  inflating: cats_dogs/val/cats/10050.jpg  
  inflating: cats_dogs/val/cats/10063.jpg  
  inflating: cats_dogs/val/cats/10064.jpg  
  inflating: cats_dogs/val/cats/10067.jpg  
  inflating: cats_dogs/val/cats/10069.jpg  
  inflating: cats_dogs/val/cats/1007.jpg  
  inflating: cats_dogs/val/cats/10070.jpg  
  inflating: cats_dogs/val/cats/10080.jpg  
  inflating: cats_dogs/val/cats/10085.jpg  
  inflating: cats_dogs/val/cats/10092.jpg  
  inflating: cats_dogs/val/cats/10096.jpg  
  inflating: cats_dogs/val/cats/101.jpg  
  inflating: cats_dogs/val/cats/1010.jpg  
  inflating: cats_dogs/val/cats/10112.jpg  
  inflating: cats_dogs/val/cat

In [2]:
!ls cats_dogs


train  val


In [3]:
from PIL import Image
import os

def clean_images(folder):
    removed = 0
    for root,_,files in os.walk(folder):
        for f in files:
            try:
                Image.open(os.path.join(root,f)).verify()
            except:
                os.remove(os.path.join(root,f))
                removed += 1
    print("Removed", removed, "corrupted images")

clean_images("/content/cats_dogs")


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Removed 0 corrupted images


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)


Using: cuda


In [5]:
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

cifar = datasets.CIFAR10("./data", train=True, download=True, transform=transform)
cifar_loader = DataLoader(cifar, batch_size=64, shuffle=True)

catsdogs = datasets.ImageFolder("/content/cats_dogs/train", transform=transform)
catsdogs_loader = DataLoader(catsdogs, batch_size=32, shuffle=True)

print("Datasets loaded")


100%|██████████| 170M/170M [00:03<00:00, 43.5MB/s]


Datasets loaded


In [6]:
class CNN(nn.Module):
    def __init__(self, act):
        super().__init__()
        self.act = act
        self.conv = nn.Sequential(
            nn.Conv2d(3,32,3), self.act, nn.MaxPool2d(2),
            nn.Conv2d(32,64,3), self.act, nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*14*14,128),
            self.act,
            nn.Linear(128,10)
        )

    def forward(self,x):
        return self.fc(self.conv(x))


In [7]:
def init_weights(m,mode):
    for l in m.modules():
        if isinstance(l,nn.Conv2d) or isinstance(l,nn.Linear):
            if mode=="xavier": nn.init.xavier_uniform_(l.weight)
            elif mode=="kaiming": nn.init.kaiming_uniform_(l.weight)
            else: nn.init.normal_(l.weight)

def train(model,loader,opt):
    model.to(device)
    lossfn = nn.CrossEntropyLoss()
    for e in range(2):
        c=t=0
        for x,y in loader:
            x=x.to(device)
            y=y.to(device)

            opt.zero_grad()
            out=model(x)
            loss=lossfn(out,y)
            loss.backward()
            opt.step()

            c+=(out.argmax(1)==y).sum().item()
            t+=y.size(0)

        print("Epoch",e+1,"Acc",round(c/t,3))


In [8]:
acts={"relu":nn.ReLU(),"tanh":nn.Tanh()}
opts={"adam":optim.Adam,"sgd":optim.SGD}

experiments=[("relu","xavier","adam"),("tanh","kaiming","sgd")]

print("CIFAR10")
for a,i,o in experiments:
    m=CNN(acts[a])
    init_weights(m,i)
    opt=opts[o](m.parameters(),lr=0.001)
    train(m,cifar_loader,opt)
    torch.save(m.state_dict(),f"cnn_cifar_{a}.pth")

print("Cats vs Dogs")
for a,i,o in experiments:
    m=CNN(acts[a])
    init_weights(m,i)
    opt=opts[o](m.parameters(),lr=0.001)
    train(m,catsdogs_loader,opt)
    torch.save(m.state_dict(),f"cnn_catdog_{a}.pth")


CIFAR10
Epoch 1 Acc 0.422
Epoch 2 Acc 0.541
Epoch 1 Acc 0.296
Epoch 2 Acc 0.383
Cats vs Dogs


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1 Acc 0.641
Epoch 2 Acc 0.746
Epoch 1 Acc 0.589
Epoch 2 Acc 0.639


In [9]:
res=models.resnet18(pretrained=True)
res.fc=nn.Linear(res.fc.in_features,10)
opt=optim.Adam(res.parameters(),lr=0.0001)

train(res,cifar_loader,opt)

torch.save(res.state_dict(),"resnet18_best.pth")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 183MB/s]


Epoch 1 Acc 0.813
Epoch 2 Acc 0.926


In [10]:
print("Experiment Finished")
print("Saved models:")
!ls *.pth


Experiment Finished
Saved models:
cnn_catdog_relu.pth  cnn_cifar_relu.pth  resnet18_best.pth
cnn_catdog_tanh.pth  cnn_cifar_tanh.pth
